In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[7]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.0,0.053609,-1.670,0.0,0.0
1,69773.0,0.753803,-1.768,0.0,0.0
2,66417.0,0.720125,-1.841,0.0,0.0
3,64182.0,0.697696,-1.908,0.0,0.0
4,63859.0,0.694454,-1.962,0.0,0.0
...,...,...,...,...,...
43819,64771.0,0.703607,2.048,0.0,0.0
43820,62259.0,0.678398,1.995,0.0,0.0
43821,63774.0,0.693601,2.107,0.0,0.0
43822,65865.0,0.714585,2.280,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[7]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_FR,Temp_FR
0,0.0,-1.670
1,69773.0,-1.768
2,66417.0,-1.841
3,64182.0,-1.908
4,63859.0,-1.962
...,...,...
43819,64771.0,2.048
43820,62259.0,1.995
43821,63774.0,2.107
43822,65865.0,2.280


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_FR,Temp_FR,Lag-2,Lag-7
0,65948.0,3.926,67795.0,0.0
1,64676.0,4.000,66811.0,69773.0
2,61551.0,4.109,64040.0,66417.0
3,60541.0,4.202,63275.0,64182.0
4,62833.0,4.258,65977.0,63859.0
...,...,...,...,...
43651,64771.0,2.048,64448.0,52597.0
43652,62259.0,1.995,62679.0,50947.0
43653,63774.0,2.107,63851.0,52867.0
43654,65865.0,2.280,65287.0,55237.0


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 14s 9ms/step - loss: 0.0075 - mape: 13374.5469 - mae: 0.0641 - val_loss: 0.0034 - val_mape: 8.1795 - val_mae: 0.0483
Epoch 2/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0048 - mape: 13565.7559 - mae: 0.0551 - val_loss: 0.0030 - val_mape: 8.1742 - val_mae: 0.0445
Epoch 3/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0042 - mape: 13203.7920 - mae: 0.0514 - val_loss: 0.0029 - val_mape: 7.3761 - val_mae: 0.0438
Epoch 4/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0034 - mape: 13207.6260 - mae: 0.0447 - val_loss: 0.0104 - val_mape: 14.9984 - val_mae: 0.0901
Epoch 5/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0024 - mape: 13508.1885 - mae: 0.0367 - val_loss: 0.0050 - val_mape: 10.1454 - val_mae: 0.0600
Epoch 6/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0020 - mape: 13309.8203 - mae: 0.0330 - val_loss: 0.0

1105/1105 [==============================] - 11s 10ms/step - loss: 8.9684e-04 - mape: 11684.2148 - mae: 0.0196 - val_loss: 0.0023 - val_mape: 6.7833 - val_mae: 0.0382
Epoch 48/100
1105/1105 [==============================] - 11s 10ms/step - loss: 9.0299e-04 - mape: 11950.4814 - mae: 0.0197 - val_loss: 0.0019 - val_mape: 6.0450 - val_mae: 0.0342
Epoch 49/100
1105/1105 [==============================] - 11s 10ms/step - loss: 8.6177e-04 - mape: 11701.7959 - mae: 0.0193 - val_loss: 0.0030 - val_mape: 7.5848 - val_mae: 0.0451
Epoch 50/100
1105/1105 [==============================] - 11s 10ms/step - loss: 9.0715e-04 - mape: 11469.9160 - mae: 0.0197 - val_loss: 0.0029 - val_mape: 7.4987 - val_mae: 0.0438
Epoch 51/100
1105/1105 [==============================] - 11s 10ms/step - loss: 8.8008e-04 - mape: 11483.4961 - mae: 0.0194 - val_loss: 0.0026 - val_mape: 7.2266 - val_mae: 0.0417
Epoch 52/100
1105/1105 [==============================] - 11s 10ms/step - loss: 8.6566e-04 - mape: 11552.8711 - m

Epoch 93/100
1105/1105 [==============================] - 8s 8ms/step - loss: 7.0429e-04 - mape: 9025.7148 - mae: 0.0173 - val_loss: 0.0026 - val_mape: 6.9938 - val_mae: 0.0389
Epoch 94/100
1105/1105 [==============================] - 8s 8ms/step - loss: 7.3156e-04 - mape: 8508.4980 - mae: 0.0178 - val_loss: 0.0030 - val_mape: 7.0033 - val_mae: 0.0408
Epoch 95/100
1105/1105 [==============================] - 9s 8ms/step - loss: 7.0100e-04 - mape: 10768.3906 - mae: 0.0175 - val_loss: 0.0026 - val_mape: 7.1999 - val_mae: 0.0415
Epoch 96/100
1105/1105 [==============================] - 9s 8ms/step - loss: 7.0819e-04 - mape: 8668.8828 - mae: 0.0174 - val_loss: 0.0032 - val_mape: 7.7451 - val_mae: 0.0441
Epoch 97/100
1105/1105 [==============================] - 9s 8ms/step - loss: 6.4794e-04 - mape: 9090.1309 - mae: 0.0168 - val_loss: 0.0030 - val_mape: 7.5314 - val_mae: 0.0432
Epoch 98/100
1105/1105 [==============================] - 9s 8ms/step - loss: 7.0399e-04 - mape: 9425.9932 - mae: 

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 959.6981873512268


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 6ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.42211035, 0.4195129 , 0.4295141 , ..., 0.73443663, 0.70760787,
       0.68535566], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.46444485, 0.4457089 , 0.44084176, ..., 0.69360148, 0.71458534,
       0.70266338])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,40939.0,36720.449219
1,39072.0,36461.621094
2,38587.0,37458.222656
3,39762.0,38927.546875
4,42435.0,42844.730469
...,...,...
4363,64771.0,68148.757812
4364,62259.0,68442.101562
4365,63774.0,67843.140625
4366,65865.0,65169.707031


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

4.961990182058484